In [9]:
import pandas as pd
import numpy as np
from loaders import *

## Eyeriss

In [10]:
show_config(ConfigRegistry.EYERISS_ARCH)
show_config(ConfigRegistry.EYERISS_COMPONENTS_DIR)
result = run_accelergy(ConfigRegistry.EYERISS_ARCH, ConfigRegistry.EYERISS_COMPONENTS_DIR)
print(result.ert)

architecture:
  # ============================================================
  # Architecture Description
  # ============================================================
  version: 0.3
  subtree:
    - name: system
      local:
        - name: DRAM
          class: DRAM
          attributes:
            type: LPDDR4
            width: 64
            block-size: 4
            word-bits: 16
      subtree:
        - name: eyeriss
          attributes:
            technology: 45nm
          local:
            - name: shared_glb
              class: smartbuffer_SRAM
              attributes:
                memory_depth: 16384
                memory_width: 64
                n_banks: 32
                block-size: 4
                word-bits: 16
                read_bandwidth: 16
                write_bandwidth: 16
            - name: DummyBuffer[0..13] # for better mapping
              class: regfile
              attributes:
                depth: 16
                width: 16
        

In [11]:
def parse_stat(layer_stat):
    """
    Total number of compute in this layer
    Number of cycles in this layer
    Energy used in this layer in Joule
    Energy / Compute in picoJoule
    """
    lines = layer_stat.split("\n")
    print('lines=',len(lines))
    if len(lines) < 25:
        print(lines)
        return (None, None, None, None)
    compute = int(lines[-18].split("=")[-1].strip())
    cycle = int(lines[-23].split(":")[-1].strip())
    energy = float(lines[-22].split(":")[-1].strip()[:-3])
    return compute, cycle, energy * 10**(-6), energy * 10**(-6)  / compute * 10**(12)

### Example layer

In [4]:
example_layer_stats, example_layer_mapping = run_timeloop_mapper(
    ConfigRegistry.EYERISS_ARCH, ConfigRegistry.EYERISS_COMPONENTS_DIR,
    ConfigRegistry.EYERISS_ARCH_CONSTRAINTS,
    ConfigRegistry.EYERISS_MAP_CONSTRAINTS,
    ConfigRegistry.VGG02_layer5, ConfigRegistry.DEFAULT_MAPPER_SETTING
)

input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 120
  Factorization options along problem dimension M = 495
  Factorization options along problem dimension R = 1
  Factorization options along problem dimension S = 1
  Factorization o

[  4] Utilization = 0.57 | pJ/Compute =   10.807 | L5[WIO] Q28 M4 C128 - L4[IO] M2 P56 M4X Q2X - L3[] Q1 M4Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] M2 
[  0] Utilization = 0.07 | pJ/Compute =  139.321 | L5[WIO] Q56 M4 - L4[IO] M16 P56 C128 M2X - L3[] Q1 M2Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] Q1 
[  5] Utilization = 0.14 | pJ/Compute =   37.150 | L5[WIO] Q7 M64 C8 - L4[IO] M2 P56 C16 Q8X - L3[] Q1 S3Y - L2[I] Q1 - L1[W] R3 - L0[O] M2 
[  2] Utilization = 0.19 | pJ/Compute =   37.164 | L5[WIO] Q7 M64 C8 - L4[IO] M2 P56 C4 Q8X - L3[] Q1 C4Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] M2 
[  7] Utilization = 0.57 | pJ/Compute =   23.291 | L5[WIO] Q7 M4 C4 - L4[IO] M2 P56 C16 Q8X - L3[] Q1 M4Y S3Y - L2[I] Q1 - L1[W] R3 C2 - L0[O] M8 
[  6] Utilization = 0.19 | pJ/Compute =   49.055 | L5[WIO] Q14 M8 C16 - L4[IO] M16 P56 C4 M2X Q4X - L3[] Q1 C2Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] Q1 
[  3] Utilization = 0.57 | pJ/Compute =   23.291 | L5[WIO] Q7 M4 C4 - L4[IO] M2 P56 C32 Q8X - L3[] Q1 M4Y S3Y - L2[I] Q1 - L1[W

[  4] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  0] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  2] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  7] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  5] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.




Summary stats for best mapping found by mapper:
  Utilization = 1.00 | pJ/Compute =    5.811


[  3] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.


In [5]:
parse_stat(example_layer_stats)

(924844032, 5505024, 0.005374359999999999, 5.8110987518379735)

### Squeeze Nets

In [6]:
squeeze_layers = ConfigRegistry.SQUEEZE_NET_LAYERS
squeeze_stat = []
for i in range(len(squeeze_layers)):
    layer_stats, layer_mapping = run_timeloop_mapper(
        ConfigRegistry.EYERISS_ARCH, ConfigRegistry.EYERISS_COMPONENTS_DIR,
        ConfigRegistry.EYERISS_ARCH_CONSTRAINTS,
        ConfigRegistry.EYERISS_MAP_CONSTRAINTS,
        squeeze_layers[i+1], ConfigRegistry.DEFAULT_MAPPER_SETTING
    )
    squeeze_stat.append(layer_stats)

input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 4
  Factorization options along problem dimension M = 210
  Factorization options along problem dimension R = 1
  Factorization options along problem dimension S = 1
  Factorization opt

[  0] Utilization = 0.21 | pJ/Compute =   11.901 | L5[WIO] Q37 - L4[IO] P111 Q3X - L3[] Q1 M4Y S3Y - L2[I] Q1 - L1[W] R3 C3 - L0[O] M16 
[  3] Utilization = 0.21 | pJ/Compute =   15.276 | L5[WIO] Q37 C3 - L4[IO] P111 Q3X - L3[] Q1 M4Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] M16 
[  2] Utilization = 0.21 | pJ/Compute =   56.243 | L5[WIO] Q37 - L4[IO] P111 C3 Q3X - L3[] Q1 M4Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] M16 
[  1] Utilization = 0.64 | pJ/Compute =   13.066 | L5[WIO] Q37 M4 - L4[IO] P111 M4X Q3X - L3[] Q1 C3Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] M4 
[  5] Utilization = 0.21 | pJ/Compute =   16.623 | L5[WIO] Q111 M4 - L4[IO] M4 P111 M4X - L3[] Q1 C3Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] Q1 
[  6] Utilization = 0.07 | pJ/Compute =  146.649 | L5[WIO] Q111 M4 - L4[IO] M4 P111 C3 M4X - L3[] Q1 S3Y - L2[I] Q1 - L1[W] R3 - L0[O] Q1 
[  4] Utilization = 0.07 | pJ/Compute =   16.623 | L5[WIO] Q111 M4 - L4[IO] M4 P111 M4X - L3[] Q1 S3Y - L2[I] Q1 - L1[W] R3 C3 - L0[O] Q1 
[  7] Utilization = 0.07 | pJ/C



Summary stats for best mapping found by mapper:
  Utilization = 0.86 | pJ/Compute =   11.930
input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 84
  Factorization options along problem dimension M = 70
  Factorization options along pr

[  2] Utilization = 0.10 | pJ/Compute =   92.989 | L5[WIO] Q55 M8 C2 - L4[IO] P55 M2X - L3[] Q1 C8Y - L2[I] Q1 - L1[W] C4 - L0[O] Q1 
[  0] Utilization = 0.05 | pJ/Compute =   92.973 | L5[WIO] Q55 M8 C2 - L4[IO] P55 M2X - L3[] Q1 C4Y - L2[I] Q1 - L1[W] C8 - L0[O] Q1 
[  4] Utilization = 0.02 | pJ/Compute =  220.194 | L5[WIO] Q55 M8 C2 - L4[IO] P55 C2 M2X - L3[] Q1 C2Y - L2[I] Q1 - L1[W] C8 - L0[O] Q1 
[  5] Utilization = 0.05 | pJ/Compute =  220.202 | L5[WIO] Q55 M8 C2 - L4[IO] P55 C2 M2X - L3[] Q1 C4Y - L2[I] Q1 - L1[W] C4 - L0[O] Q1 
[  7] Utilization = 0.10 | pJ/Compute =  220.218 | L5[WIO] Q55 M8 C2 - L4[IO] P55 C2 M2X - L3[] Q1 C8Y - L2[I] Q1 - L1[W] C2 - L0[O] Q1 
[  6] Utilization = 0.24 | pJ/Compute =   47.788 | L5[WIO] Q11 C4 - L4[IO] M2 P55 C4 M2X Q5X - L3[] Q1 M4Y - L2[I] Q1 - L1[W] C4 - L0[O] Q1 
[  1] Utilization = 0.01 | pJ/Compute =  168.551 | L5[WIO] Q55 M2 C16 - L4[IO] M2 P55 C4 - L3[] Q1 - L2[I] Q1 - L1[W] Q1 - L0[O] M4 
[  3] Utilization = 0.48 | pJ/Compute =   47.79



Summary stats for best mapping found by mapper:
  Utilization = 0.52 | pJ/Compute =   18.874
input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 35
  Factorization options along problem dimension M = 210
  Factorization options along p

...

[  6] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  2] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  3] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.




Summary stats for best mapping found by mapper:
  Utilization = 0.93 | pJ/Compute =    6.789
input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 220
  Factorization options along problem dimension M = 1225
  Factorization options along

[  2] Utilization = 0.06 | pJ/Compute =  134.998 | L5[WIO] Q13 M20 - L4[IO] P13 C512 M5X - L3[] Q1 M2Y - L2[I] Q1 - L1[W] Q1 - L0[O] M5 
[  0] Utilization = 0.01 | pJ/Compute =  168.107 | L5[WIO] Q13 M100 C4 - L4[IO] M10 P13 C64 - L3[] Q1 C2Y - L2[I] Q1 - L1[W] Q1 - L0[O] Q1 
[  5] Utilization = 0.01 | pJ/Compute =  168.106 | L5[WIO] Q13 M100 C4 - L4[IO] M10 P13 C32 - L3[] Q1 - L2[I] Q1 - L1[W] C4 - L0[O] Q1 
[  1] Utilization = 0.01 | pJ/Compute =  168.106 | L5[WIO] Q13 M100 C4 - L4[IO] M10 P13 C16 - L3[] Q1 - L2[I] Q1 - L1[W] C8 - L0[O] Q1 
[  6] Utilization = 0.01 | pJ/Compute =  168.107 | L5[WIO] Q13 M100 C4 - L4[IO] M10 P13 C32 - L3[] Q1 C2Y - L2[I] Q1 - L1[W] C2 - L0[O] Q1 
[  4] Utilization = 0.45 | pJ/Compute =   28.080 | L5[WIO] M50 C8 - L4[IO] M4 P13 C4 Q13X - L3[] Q1 M5Y C2Y - L2[I] Q1 - L1[W] C8 - L0[O] Q1 
[  3] Utilization = 0.02 | pJ/Compute =  168.110 | L5[WIO] Q13 M100 C4 - L4[IO] M10 P13 C16 - L3[] Q1 C4Y - L2[I] Q1 - L1[W] C2 - L0[O] Q1 
[  7] Utilization = 0.02 | pJ

[  3] Utilization = 0.77 | pJ/Compute =   18.023 | L5[WIO] M5 C4 - L4[IO] M8 P13 C8 Q13X - L3[] Q1 M5Y C2Y - L2[I] Q1 - L1[W] C8 - L0[O] M5 
[  5] Utilization = 0.77 | pJ/Compute =   18.610 | L5[WIO] M10 C2 - L4[IO] M4 P13 C16 Q13X - L3[] Q1 M5Y C2Y - L2[I] Q1 - L1[W] C8 - L0[O] M5 
[  5] Utilization = 0.77 | pJ/Compute =   17.778 | L5[WIO] M4 C8 - L4[IO] M5 P13 C8 Q13X - L3[] Q1 M5Y C2Y - L2[I] Q1 - L1[W] C4 - L0[O] M10 
[  4] Utilization = 0.77 | pJ/Compute =   18.659 | L5[WIO] M10 C8 - L4[IO] M2 P13 C4 Q13X - L3[] Q1 M5Y C2Y - L2[I] Q1 - L1[W] C8 - L0[O] M10 
[  7] Utilization = 0.77 | pJ/Compute =   17.877 | L5[WIO] M4 C4 - L4[IO] M10 P13 C8 Q13X - L3[] Q1 M5Y C2Y - L2[I] Q1 - L1[W] C8 - L0[O] M5 
[  4] Utilization = 0.77 | pJ/Compute =   17.877 | L5[WIO] M4 C4 - L4[IO] M10 P13 C64 Q13X - L3[] Q1 M5Y C2Y - L2[I] Q1 - L1[W] Q1 - L0[O] M5 
[  2] Utilization = 0.77 | pJ/Compute =   17.482 | L5[WIO] M4 C4 - L4[IO] M5 P13 C32 Q13X - L3[] Q1 M5Y C2Y - L2[I] Q1 - L1[W] C2 - L0[O] M10 
[  



Summary stats for best mapping found by mapper:
  Utilization = 0.77 | pJ/Compute =    9.245


In [7]:
squeeze_total_compute = 0
squeeze_total_cycle = 0
squeeze_total_energy = 0

for stat in squeeze_stat:
    compute, cycle, energy, energy_per_compute = parse_stat(stat)
    squeeze_total_compute += compute
    squeeze_total_cycle += cycle
    squeeze_total_energy += energy

print(f"Squeeze Net Total Compute:", squeeze_total_compute)
print(f"Squeeze Net Total Cycle:", squeeze_total_cycle)
print(f"Squeeze Net Total Energy:", squeeze_total_energy)
print(f"Squeeze Net Energy / Computer pJ / Compute", squeeze_total_energy / squeeze_total_compute * 10 ** (12))

Squeeze Net Total Compute: 349151936
Squeeze Net Total Cycle: 2626188
Squeeze Net Total Energy: 0.00307392
Squeeze Net Energy / Computer pJ / Compute 8.803960920898344


In [11]:
import json
with open("eyeriss-squeezenet.json", "w") as f:
    json.dump({"stats": squeeze_stat}, f)

### Mobile Nets

In [24]:
mobile_layers = ConfigRegistry.MOBILE_NET_LAYERS
mobile_stat = []
for i in range(len(mobile_layers)):
    layer_stats, layer_mapping = run_timeloop_mapper(
        ConfigRegistry.EYERISS_ARCH, ConfigRegistry.EYERISS_COMPONENTS_DIR,
        ConfigRegistry.EYERISS_ARCH_CONSTRAINTS,
        ConfigRegistry.EYERISS_MAP_CONSTRAINTS,
        mobile_layers[i+1], ConfigRegistry.DEFAULT_MAPPER_SETTING
    )
    mobile_stat.append(layer_stats)

input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 4
  Factorization options along problem dimension M = 126
  Factorization options along problem dimension R = 1
  Factorization options along problem dimension S = 1
  Factorization opt

[  0] Utilization = 0.04 | pJ/Compute =   20.236 | L5[WIO] Q112 - L4[IO] M16 P112 - L3[] Q1 M2Y S3Y - L2[I] Q1 - L1[W] R3 C3 - L0[O] Q1 
[  2] Utilization = 0.04 | pJ/Compute =  151.805 | L5[WIO] Q112 - L4[IO] M16 P112 C3 - L3[] Q1 M2Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] Q1 
[  3] Utilization = 0.04 | pJ/Compute =   24.073 | L5[WIO] Q112 C3 - L4[IO] M16 P112 - L3[] Q1 M2Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] Q1 
[  1] Utilization = 0.21 | pJ/Compute =   14.586 | L5[WIO] Q28 - L4[IO] M4 P112 Q4X - L3[] Q1 C3Y S3Y - L2[I] Q1 - L1[W] R3 - L0[O] M8 
[  7] Utilization = 0.04 | pJ/Compute =   24.073 | L5[WIO] Q112 C3 - L4[IO] M16 P112 M2X - L3[] Q1 S3Y - L2[I] Q1 - L1[W] R3 - L0[O] Q1 
[  6] Utilization = 0.04 | pJ/Compute =  151.805 | L5[WIO] Q112 - L4[IO] M16 P112 C3 M2X - L3[] Q1 S3Y - L2[I] Q1 - L1[W] R3 - L0[O] Q1 
[  4] Utilization = 0.04 | pJ/Compute =   20.236 | L5[WIO] Q112 - L4[IO] M16 P112 M2X - L3[] Q1 S3Y - L2[I] Q1 - L1[W] R3 C3 - L0[O] Q1 
[  5] Utilization = 0.11 | pJ/Compute =   



Summary stats for best mapping found by mapper:
  Utilization = 1.00 | pJ/Compute =   13.153
input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
[ERROR] 2023-05-05 16:03:56,313 - pytimeloop.app.mapper - Could not find timeloop-mapper.stats.txt
[ERROR] 2023-05-05 16:03:56,314 - pytimeloop.app.mapper - Could not find timeloop-mapp

ERROR: parsing permutation: NMCPQRS: dimension M not found in problem shape.


Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 56
  Factorization options along problem dimension M = 70
  Factorization options along problem dimension R = 1
  Factorization options along problem dimension S = 1
  Factorization options along problem dimension N = 1
  Factorization options along problem dimension P = 1
  Factorization options along problem dimension Q = 10
Mapspace Dimension [IndexFactorization] Size: 39200
Mapspace Dimension [LoopPermutation] Size: 1
Mapspace Dimension [Spatial] Size: 1
Mapspace Dimension [DatatypeBypass] Size: 1
Mapspace split! Per-split Mapping Dimension [IndexFactorization] Size: 4900 Residue: 0
Mapspace construction

...

[  2] Utilization = 0.17 | pJ/Compute =   28.541 | L5[WIO] M20 C2 - L4[IO] M4 P7 C32 M2X Q7X - L3[] Q1 M2Y - L2[I] Q1 - L1[W] C5 - L0[O] M4 
[  6] Utilization = 0.83 | pJ/Compute =   28.545 | L5[WIO] M20 C2 - L4[IO] M4 P7 C16 M2X Q7X - L3[] Q1 M2Y C5Y - L2[I] Q1 - L1[W] C2 - L0[O] M4 
[  5] Utilization = 0.33 | pJ/Compute =   28.542 | L5[WIO] M20 C2 - L4[IO] M4 P7 C16 M2X Q7X - L3[] Q1 M2Y C2Y - L2[I] Q1 - L1[W] C5 - L0[O] M4 
[  4] Utilization = 0.17 | pJ/Compute =   28.541 | L5[WIO] M20 C2 - L4[IO] M4 P7 C16 M2X Q7X - L3[] Q1 M2Y - L2[I] Q1 - L1[W] C10 - L0[O] M4 
[  0] Utilization = 0.10 | pJ/Compute =  136.713 | L5[WIO] Q7 M5 C2 - L4[IO] M16 P7 C20 M8X - L3[] Q1 M2Y - L2[I] Q1 - L1[W] C8 - L0[O] Q1 
[  3] Utilization = 0.83 | pJ/Compute =   28.545 | L5[WIO] M20 C2 - L4[IO] M4 P7 C32 M2X Q7X - L3[] Q1 M2Y C5Y - L2[I] Q1 - L1[W] Q1 - L0[O] M4 
[  7] Utilization = 0.06 | pJ/Compute =  135.339 | L5[WIO] Q7 M2 C4 - L4[IO] M4 P7 C80 M5X - L3[] Q1 M2Y - L2[I] Q1 - L1[W] Q1 - L0[O] M16 
[ 



Summary stats for best mapping found by mapper:
  Utilization = 0.83 | pJ/Compute =    9.671
input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 660
  Factorization options along problem dimension M = 1225
  Factorization options along

[  1] Utilization = 0.05 | pJ/Compute =  155.143 | L5[WIO] M125 C8 - L4[IO] C80 M2X - L3[] Q1 M4Y - L2[I] Q1 - L1[W] C2 - L0[O] Q1 
[  7] Utilization = 0.03 | pJ/Compute =  143.772 | L5[WIO] Q1 - L4[IO] M500 C128 - L3[] Q1 C5Y - L2[I] Q1 - L1[W] C2 - L0[O] M2 
[  0] Utilization = 0.06 | pJ/Compute =  143.772 | L5[WIO] Q1 - L4[IO] M500 C128 - L3[] Q1 C10Y - L2[I] Q1 - L1[W] Q1 - L0[O] M2 
[  5] Utilization = 0.01 | pJ/Compute =  143.772 | L5[WIO] Q1 - L4[IO] M500 C128 - L3[] Q1 - L2[I] Q1 - L1[W] C10 - L0[O] M2 
[  2] Utilization = 0.10 | pJ/Compute =  155.143 | L5[WIO] M125 C8 - L4[IO] C80 M2X - L3[] Q1 M4Y C2Y - L2[I] Q1 - L1[W] Q1 - L0[O] Q1 
[  6] Utilization = 0.01 | pJ/Compute =  143.772 | L5[WIO] Q1 - L4[IO] M500 C128 - L3[] Q1 C2Y - L2[I] Q1 - L1[W] C5 - L0[O] M2 
[  3] Utilization = 0.05 | pJ/Compute =  155.143 | L5[WIO] M125 C8 - L4[IO] C20 M2X - L3[] Q1 M4Y - L2[I] Q1 - L1[W] C8 - L0[O] Q1 
[  4] Utilization = 0.30 | pJ/Compute =  137.459 | L5[WIO] M2 C80 - L4[IO] M10 C8 M10X



Summary stats for best mapping found by mapper:
  Utilization = 0.60 | pJ/Compute =  134.502


[  1] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.


In [26]:
mobile_total_compute = 0
mobile_total_cycle = 0
mobile_total_energy = 0

for stat in mobile_stat:
    compute, cycle, energy, energy_per_compute = parse_stat(stat)
    if compute is not None:
        mobile_total_compute += compute
        mobile_total_cycle += cycle
        mobile_total_energy += energy

print(f"Mobile Net Total Compute:", mobile_total_compute)
print(f"Mobile Net Total Cycle:", mobile_total_cycle)
print(f"Mobile Net Total Energy:", mobile_total_energy)
print(f"Mobile Net Energy / Computer pJ / Compute", mobile_total_energy / mobile_total_compute * 10 ** (12))

lines= 851
lines= 1
['']
lines= 849
lines= 852
lines= 1
['']
lines= 851
lines= 851
lines= 1
['']
lines= 851
lines= 851
lines= 1
['']
lines= 849
lines= 850
lines= 1
['']
lines= 852
lines= 850
lines= 1
['']
lines= 852
lines= 850
lines= 1
['']
lines= 850
lines= 852
lines= 1
['']
lines= 852
lines= 852
lines= 1
['']
lines= 852
lines= 852
lines= 1
['']
lines= 852
lines= 852
lines= 1
['']
lines= 850
lines= 851
lines= 1
['']
lines= 850
lines= 851
lines= 1
['']
lines= 850
lines= 851
lines= 1
['']
lines= 852
lines= 852
lines= 1
['']
lines= 852
lines= 852
lines= 1
['']
lines= 852
lines= 852
lines= 1
['']
lines= 851
lines= 851
lines= 849
Mobile Net Total Compute: 280057856
Mobile Net Total Cycle: 1763584
Mobile Net Total Energy: 0.0037043699999999994
Mobile Net Energy / Computer pJ / Compute 13.227159748020064


In [25]:
import json
with open("eyeriss-mobilenet.json", "w") as f:
    json.dump({"stats": mobile_stat}, f)

### Dense Net

In [12]:
dense_layers = ConfigRegistry.DENSE_NET_LAYERS 
dense_stat = []
for i in range(len(dense_layers)):
    layer_stats, layer_mapping = run_timeloop_mapper(
        ConfigRegistry.EYERISS_ARCH, ConfigRegistry.EYERISS_COMPONENTS_DIR,
        ConfigRegistry.EYERISS_ARCH_CONSTRAINTS,
        ConfigRegistry.EYERISS_MAP_CONSTRAINTS,
        dense_layers[i+1], ConfigRegistry.DEFAULT_MAPPER_SETTING
    )
    dense_stat.append(layer_stats)

input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 4
  Factorization options along problem dimension M = 210
  Factorization options along problem dimension R = 1
  Factorization options along problem dimension S = 1
  Factorization opt

[  2] Utilization = 0.33 | pJ/Compute =  136.708 | L5[WIO] Q112 - L4[IO] M4 P112 C3 M8X - L3[] Q1 S7Y - L2[I] Q1 - L1[W] R7 - L0[O] M2 
[  7] Utilization = 0.33 | pJ/Compute =    9.913 | L5[WIO] Q56 M4 C3 - L4[IO] M4 P112 M4X Q2X - L3[] Q1 S7Y - L2[I] Q1 - L1[W] R7 - L0[O] Q1 
[  3] Utilization = 0.33 | pJ/Compute =    8.382 | L5[WIO] Q112 C3 - L4[IO] M4 P112 M8X - L3[] Q1 S7Y - L2[I] Q1 - L1[W] R7 - L0[O] M2 
[  6] Utilization = 0.33 | pJ/Compute =   76.537 | L5[WIO] Q56 M4 - L4[IO] M4 P112 C3 M4X Q2X - L3[] Q1 S7Y - L2[I] Q1 - L1[W] R7 - L0[O] Q1 
[  7] Utilization = 0.58 | pJ/Compute =    9.922 | L5[WIO] Q8 M4 C3 - L4[IO] M8 P112 Q14X - L3[] Q1 S7Y - L2[I] Q1 - L1[W] R7 - L0[O] M2 
[  6] Utilization = 0.33 | pJ/Compute =   40.026 | L5[WIO] Q28 - L4[IO] M2 P112 C3 M2X Q4X - L3[] Q1 S7Y - L2[I] Q1 - L1[W] R7 - L0[O] M16 
[  2] Utilization = 0.33 | pJ/Compute =   25.834 | L5[WIO] Q14 M8 - L4[IO] P112 C3 Q8X - L3[] Q1 S7Y - L2[I] Q1 - L1[W] R7 - L0[O] M8 
[  3] Utilization = 0.58 | pJ/C



Summary stats for best mapping found by mapper:
  Utilization = 0.58 | pJ/Compute =    6.793
input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 84
  Factorization options along problem dimension M = 330
  Factorization options along p

[  6] Utilization = 0.19 | pJ/Compute =   22.466 | L5[WIO] Q28 M4 C16 - L4[IO] P56 M4X Q2X - L3[] Q1 C4Y - L2[I] Q1 - L1[W] Q1 - L0[O] M8 
[  4] Utilization = 0.05 | pJ/Compute =   22.289 | L5[WIO] Q28 M4 C16 - L4[IO] P56 M4X Q2X - L3[] Q1 - L2[I] Q1 - L1[W] C4 - L0[O] M8 
[  2] Utilization = 0.05 | pJ/Compute =   79.655 | L5[WIO] Q28 M4 C4 - L4[IO] P56 C8 M4X Q2X - L3[] Q1 - L2[I] Q1 - L1[W] C2 - L0[O] M8 
[  1] Utilization = 0.05 | pJ/Compute =   79.655 | L5[WIO] Q28 M4 C4 - L4[IO] P56 C16 M4X Q2X - L3[] Q1 - L2[I] Q1 - L1[W] Q1 - L0[O] M8 
[  7] Utilization = 0.05 | pJ/Compute =   86.704 | L5[WIO] Q28 M4 C16 - L4[IO] P56 C2 M4X Q2X - L3[] Q1 - L2[I] Q1 - L1[W] C2 - L0[O] M8 
[  3] Utilization = 0.10 | pJ/Compute =   79.667 | L5[WIO] Q28 M4 C4 - L4[IO] P56 C8 M4X Q2X - L3[] Q1 C2Y - L2[I] Q1 - L1[W] Q1 - L0[O] M8 
[  5] Utilization = 0.10 | pJ/Compute =   22.348 | L5[WIO] Q28 M4 C16 - L4[IO] P56 M4X Q2X - L3[] Q1 C2Y - L2[I] Q1 - L1[W] C2 - L0[O] M8 
[  0] Utilization = 0.19 | pJ/Com

[  7] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  5] Utilization = 0.67 | pJ/Compute =   12.199 | L5[WIO] Q8 M2 C4 - L4[IO] M2 P56 M2X Q7X - L3[] Q1 C8Y - L2[I] Q1 - L1[W] C2 - L0[O] M16 
[  3] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  0] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  4] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  5] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.


...

[  7] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.




Summary stats for best mapping found by mapper:
  Utilization = 1.00 | pJ/Compute =    8.849
input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 286
  Factorization options along problem dimension M = 1225
  Factorization options along

[  5] Utilization = 0.05 | pJ/Compute =  155.175 | L5[WIO] M125 C256 - L4[IO] C4 M2X - L3[] Q1 M4Y - L2[I] Q1 - L1[W] Q1 - L0[O] Q1 
[  4] Utilization = 0.10 | pJ/Compute =  155.175 | L5[WIO] M125 C256 - L4[IO] C2 M2X - L3[] Q1 M4Y C2Y - L2[I] Q1 - L1[W] Q1 - L0[O] Q1 
[  3] Utilization = 0.05 | pJ/Compute =  155.175 | L5[WIO] M125 C256 - L4[IO] C2 M2X - L3[] Q1 M4Y - L2[I] Q1 - L1[W] C2 - L0[O] Q1 
[  7] Utilization = 0.60 | pJ/Compute =  135.726 | L5[WIO] M2 C16 - L4[IO] M10 C8 M10X - L3[] Q1 M5Y C2Y - L2[I] Q1 - L1[W] C4 - L0[O] Q1 
[  6] Utilization = 0.30 | pJ/Compute =  135.722 | L5[WIO] M2 C16 - L4[IO] M10 C8 M10X - L3[] Q1 M5Y - L2[I] Q1 - L1[W] C8 - L0[O] Q1 
[  1] Utilization = 0.02 | pJ/Compute =  143.804 | L5[WIO] Q1 - L4[IO] M500 C256 - L3[] Q1 C4Y - L2[I] Q1 - L1[W] Q1 - L0[O] M2 
[  2] Utilization = 0.30 | pJ/Compute =  135.722 | L5[WIO] M2 C16 - L4[IO] M10 C16 M10X - L3[] Q1 M5Y - L2[I] Q1 - L1[W] C4 - L0[O] Q1 
[  3] Utilization = 0.60 | pJ/Compute =  135.726 | L5[WIO]



Summary stats for best mapping found by mapper:
  Utilization = 0.60 | pJ/Compute =  134.460


[  0] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.
[  7] STATEMENT: 500 suboptimal mappings found since the last upgrade, terminating search.


In [13]:
dense_total_compute = 0
dense_total_cycle = 0
dense_total_energy = 0

for stat in dense_stat:
    compute, cycle, energy, energy_per_compute = parse_stat(stat)
    dense_total_compute += compute
    dense_total_cycle += cycle
    dense_total_energy += energy

print(f"Dense Net Total Compute:", dense_total_compute)
print(f"Dense Net Total Cycle:", dense_total_cycle)
print(f"Dense Net Total Energy:", dense_total_energy)
print(f"Dense Net Energy / Computer pJ / Compute", dense_total_energy / dense_total_compute * 10 ** (12))

lines= 851
lines= 852
lines= 852
lines= 853
lines= 852
lines= 852
lines= 852
lines= 851
lines= 852
lines= 851
lines= 852
lines= 851
lines= 852
lines= 851
lines= 852
lines= 852
lines= 852
lines= 852
lines= 852
lines= 852
lines= 851
lines= 852
lines= 852
lines= 852
lines= 852
lines= 852
lines= 853
lines= 852
lines= 851
lines= 852
lines= 851
lines= 852
lines= 851
lines= 852
lines= 851
lines= 852
lines= 851
lines= 852
lines= 851
lines= 850
lines= 851
lines= 850
lines= 851
lines= 852
lines= 851
lines= 851
lines= 851
lines= 852
lines= 851
lines= 850
lines= 851
lines= 851
lines= 851
lines= 852
lines= 851
lines= 850
lines= 851
lines= 849
lines= 851
lines= 850
lines= 851
lines= 849
lines= 851
lines= 850
lines= 851
lines= 851
lines= 851
lines= 851
lines= 851
lines= 849
lines= 851
lines= 849
lines= 851
lines= 849
lines= 851
lines= 851
lines= 851
lines= 850
lines= 851
lines= 850
lines= 851
lines= 849
lines= 851
lines= 850
lines= 851
lines= 849
lines= 851
lines= 851
lines= 851
lines= 851
lines= 851

In [14]:
import json
with open("eyeriss-densenet.json", "w") as f:
    json.dump({"stats": dense_stat}, f)